# Surreal Number python lists
Surreal numbers are explained here: https://en.wikipedia.org/wiki/Surreal_number

This notebook explains how to create and manipulate surreal numbers using python's native list type.

Author: Jeff Anderson - 2020-03-10

### Surreal numbers as python lists
Generate some surreal numbers as python lists...

In [1]:
void = ()
zero = (void,void)
one  = (zero,void)
two  = (one ,void)
half = (zero,one )
neg  = (void,zero)
ntwo = (void,neg )
nhalf= (neg ,zero)
print('zero:',zero)
print('one:',one)
print('two:',two)
print('negative two:',ntwo)
print('one half:',half)

zero: ((), ())
one: (((), ()), ())
two: ((((), ()), ()), ())
negative two: ((), ((), ((), ())))
one half: (((), ()), (((), ()), ()))


Now we can compare these:

In [2]:
one > zero

True

This works. One really is greater than zero. Good so far.

Test another obvious truth...

In [3]:
half > two

False

THIS IS INCORRECT. One half is less than two.

It shows that although python lists can hold surreal number representations, they will not compare directly. Or at least the comparison result will not match the expected result if we compared the numbers by which we labelled them.

Python will NOT compare these as surreal numbers should be.

To correct this, a simple python routine can be used.

### Surreal number comparison routine

In [24]:
def le (x,y): return x == y or not (x[0] and le(y,x[0]) or y[1] and le(y[1],x) )

Using the above routine, the surreal list representations can be compared to see which represents a larger or smaller number.

In [25]:
print('It should be True that one half is less or equal to one. It says:', le(half,one))

It should be True that one half is less or equal to one. It says: True


The above comparison is now corrected. But what about all the other comparisons.
This is hardly satisfying evidence that the routine can compare surreal representations.

To gather more analysis, we generate a set of Surreal numbers and test in code below.

### Surreal labels

In [26]:
from fractions import Fraction
def canal (r=[Fraction(0,1)]):
    yield r[0]
    while 1:
        yield r[0] - 1
        rn = [r[0]]
        for n in r[1:]:
            m = (rn[-1]+n)/2
            yield m
            rn.extend((m,n))
        yield rn[-1]+1
        r = [rn[0]-1] + rn + [rn[-1]+1]

Which generates numbers in the order that surreal numbers are born:

In [27]:
birth = canal()
for i in range(2**4):
    print('{:5} : {:5}'.format(i,str(next(birth))))

    0 : 0    
    1 : -1   
    2 : 1    
    3 : -2   
    4 : -1/2 
    5 : 1/2  
    6 : 2    
    7 : -3   
    8 : -3/2 
    9 : -3/4 
   10 : -1/4 
   11 : 1/4  
   12 : 3/4  
   13 : 3/2  
   14 : 3    
   15 : -4   


Next from a simple nucleus, numbers sprout which we cleave off in this code:

### Surreal representation generator

In [28]:
def cleave (nucleus=[zero]):
    l = nucleus
    yield l[-1]
    while 1:
        cnt = 0
        nl  = []
        for s in l:
            for n in [(s[0],s),(s,s[1])]:
                yield n
                nl = nl + [n]
        l = nl



In [29]:
sprout = cleave()
for i in range(2**4):
    print('{:5} : {:5}'.format(i,str(next(sprout))))

    0 : ((), ())
    1 : ((), ((), ()))
    2 : (((), ()), ())
    3 : ((), ((), ((), ())))
    4 : (((), ((), ())), ((), ()))
    5 : (((), ()), (((), ()), ()))
    6 : ((((), ()), ()), ())
    7 : ((), ((), ((), ((), ()))))
    8 : (((), ((), ((), ()))), ((), ((), ())))
    9 : (((), ((), ())), (((), ((), ())), ((), ())))
   10 : ((((), ((), ())), ((), ())), ((), ()))
   11 : (((), ()), (((), ()), (((), ()), ())))
   12 : ((((), ()), (((), ()), ())), (((), ()), ()))
   13 : ((((), ()), ()), ((((), ()), ()), ()))
   14 : (((((), ()), ()), ()), ())
   15 : ((), ((), ((), ((), ((), ())))))


### Surreal creation set
labels and numbers are connected with this code:

In [30]:
def creation (days=7):
    birth    = canal()
    sprout   = cleave()
    universe = {}
    for i in range(2**days-1):
        universe[next(birth)] = next(sprout)
    return universe

In [31]:
surreals = creation(days=4)
for s in surreals:
    print('{:>5} : {}'.format(str(s),surreals[s]))

    0 : ((), ())
    1 : (((), ()), ())
    2 : ((((), ()), ()), ())
    3 : (((((), ()), ()), ()), ())
 -1/2 : (((), ((), ())), ((), ()))
  3/2 : ((((), ()), ()), ((((), ()), ()), ()))
  1/4 : (((), ()), (((), ()), (((), ()), ())))
  1/2 : (((), ()), (((), ()), ()))
   -2 : ((), ((), ((), ())))
  3/4 : ((((), ()), (((), ()), ())), (((), ()), ()))
 -3/2 : (((), ((), ((), ()))), ((), ((), ())))
 -3/4 : (((), ((), ())), (((), ((), ())), ((), ())))
   -3 : ((), ((), ((), ((), ()))))
   -1 : ((), ((), ()))
 -1/4 : ((((), ((), ())), ((), ())), ((), ()))


### Surreal comparison tests
Now we do a comparison of each of these with one another while using the numberic comparison of label names to validate our surreal comparison.

In [18]:
def ge (a,b) : return le(b,a)
def lt (a,b) : return not le(b,a)
def eq (a,b) : return le(b,a) and le(a,b)
def gt (a,b) : return not le(a,b)
def comparison(days=4, verbose=True):
        s = creation(days=days)
        print('\ncompare {}×{} numbers in 6 ways for {} comparisons.'.format(len(s),len(s),6*len(s)**2))

        for i in s:
            for j in s:
                if verbose: print('check < <= == => > relations between {} and {}'.format(i,j)) 
                assert lt(s[i],s[j]) is (i< j), 'Error: {} <  {} ≠ {}'.format(i,j,i< j)
                assert le(s[i],s[j]) is (i<=j), 'Error: {} <= {} ≠ {}'.format(i,j,i<=j)
                assert eq(s[i],s[j]) is (i==j), 'Error: {} == {} ≠ {}'.format(i,j,i==j)
                assert ge(s[i],s[j]) is (i>=j), 'Error: {} >= {} ≠ {}'.format(i,j,i>=j)
                assert gt(s[i],s[j]) is (i> j), 'Error: {} >  {} ≠ {}'.format(i,j,i> j)
        print('\ndone.\n\ncomparison success: these numbers operate according to their given labels.')

comparison()



compare 15×15 numbers in 6 ways for 1350 comparisons.
check < <= == => > relations between 0 and 0
check < <= == => > relations between 0 and 1
check < <= == => > relations between 0 and 2
check < <= == => > relations between 0 and 3
check < <= == => > relations between 0 and -1/2
check < <= == => > relations between 0 and 3/2
check < <= == => > relations between 0 and 1/4
check < <= == => > relations between 0 and 1/2
check < <= == => > relations between 0 and -2
check < <= == => > relations between 0 and 3/4
check < <= == => > relations between 0 and -3/2
check < <= == => > relations between 0 and -3/4
check < <= == => > relations between 0 and -3
check < <= == => > relations between 0 and -1
check < <= == => > relations between 0 and -1/4
check < <= == => > relations between 1 and 0
check < <= == => > relations between 1 and 1
check < <= == => > relations between 1 and 2
check < <= == => > relations between 1 and 3
check < <= == => > relations between 1 and -1/2
check < <= == => > 

A much larger comparison:

In [23]:
comparison(days=10,verbose=False)


compare 1023×1023 numbers in 6 ways for 6279174 comparisons.

done.

comparison success: these numbers operate according to their given labels.


### Conclusion
Analytic tests show that Surreal representations can be held using the native list (or tuple) type and compared with in one line of code.